In [111]:
%aimport -tf
%aimport data
%aimport model
%autoreload 1

In [112]:
import tensorflow as tf
import pandas as pd
import numpy as np
import data
import model

In [113]:
tf.logging.set_verbosity(tf.logging.INFO)

In [114]:
ds = data.load_reddit_data(
    max_document_length=30,
    min_frequency=500,
    test_size=1000)

In [115]:
data.print_dataset_stats(ds)

Vocabulary Size: 2345
Train Data Shape: (389367, 30)
Dev Data Shape (1000, 30)


In [137]:
BATCH_SIZE=128
train_input_fn, dev_input_fn = data.create_train_dev_input_fns(ds, batch_size=BATCH_SIZE)

In [140]:
def simple_rnn(sequences, sequence_lengths):
    cell = tf.nn.rnn_cell.GRUCell(64)
    x_list = tf.unpack(sequences, axis=1)
    return tf.nn.rnn(cell, sequences, dtype=tf.float32, sequence_length=sequence_lengths)

model_fn = model.create_lm(
    vocab_size=len(ds.vocab.vocabulary_),
    embedding_dim=64,
    rnn_fn=simple_rnn)

In [139]:
estimator = tf.contrib.learn.Estimator(model_fn=model_fn)
dev_monitor = tf.contrib.learn.monitors.ValidationMonitor(input_fn=dev_input_fn, every_n_steps=1000)
estimator.fit(input_fn=train_input_fn, steps=5000, monitors=[dev_monitor])

INFO:tensorflow:Create CheckpointSaver
INFO:tensorflow:Step 1: loss = 5.22488
INFO:tensorflow:Step 101: loss = 3.42184
INFO:tensorflow:Step 201: loss = 3.50796


KeyboardInterrupt: 

In [126]:
MAX_LEN=20
sentence = "I"
for i in range(MAX_LEN):
    print(sentence)
    x = np.array(list(ds.vocab.transform([sentence])))
    x_len = [i + 1]
    probs = estimator.predict(input_fn=lambda: {"x": sample, "x_len": [1]})
    next_word_idx = np.argmax(probs[i], axis=1)
    next_word = next(ds.vocab.reverse([next_word_idx]))
    sentence = sentence + " " + next_word
    if next_word == "EOS":
        break
    if i == MAX_LEN:
        break
    

I


NotFittedError: Couldn't find trained model at /var/folders/d3/jc_t7rns3_qf61016_0zhrh800c0pj/T/tmpc38nxszj.